In [57]:
from pytket.extensions.nexus import Nexus, QuantinuumConfig
#from pytket.extensions.quantinuum import QuantinuumBackend
from pytket.extensions.nexus.backends import NexusBackend
from pytket.backends.resulthandle import ResultHandle
from pytket import Circuit
from pytket.unit_id import BitRegister
from pytket.circuit.display import render_circuit_jupyter
import networkx as nx
import matplotlib.pyplot as plt
import math as ma 
import numpy as np
import random as rm
import pandas as pd
import re
import copy
import warnings
import sympy as sy
import itertools
import ast
from datetime import datetime
import sys
sys.path.append('/home/jovyan/BenchmarkingQuantinuumH1/src')
today_date = datetime.today().strftime('%Y%m%d')
data_path = '/home/jovyan/BenchmarkingQuantinuumH1/data/'
file_name = 'circuit_01_output_per_shots.csv'
data_name = data_path + today_date + '_'+ file_name

import helper_functions as hf
import circuits_version_02 as cv2

In [8]:
#my_new_quantinuum_exp = Nexus().get_project_by_name("UBQC")
#configuration = QuantinuumConfig(device_name="H1-1E", user_group="Default")
#backend = NexusBackend(configuration, my_new_quantinuum_exp)

from pytket.extensions.quantinuum import QuantinuumBackend, QuantinuumAPIOffline
api_offline = QuantinuumAPIOffline()
backend = QuantinuumBackend(device_name="H1-1LE", api_handler = api_offline)

In [9]:
n_shots = 10

In [10]:
circuit = cv2.circuit_01()
compiled_circuit = backend.get_compiled_circuit(circuit)

In [11]:
handle = backend.process_circuit(compiled_circuit, n_shots=n_shots)

In [7]:
backend.cost(compiled_circuit, n_shots = 10, syntax_checker = "H1-1SC")

NexusCircuit(id=b8a08828-2b89-418f-88b0-db6951633bda, submitted_time=2024-04-24 12:05:35.557903+00:00, project_name=UBQC, project_id=6563e6c1-b062-4a9f-b590-31804ac649e2)[PhasedX(0.5, 1.5) q[0]; Measure q[0] --> Z_condition_reg[0]; Reset q[0]; PhasedX(0.5, 1.5) q[0]; Measure q[0] --> X_condition_reg[0]; Reset q[0]; ClassicalExpBox X_condition_reg[0], Z_condition_reg[0], tk_SCRATCH_BIT[0]; IF ([X_condition_reg[0]] == 1) THEN Phase(0.5); PhasedX(0.5, 1.5) q[0]; IF ([tk_SCRATCH_BIT[0]] == 1) THEN Phase(1.75); Measure q[0] --> Z_condition_gate_reg[0]; Reset q[0]; IF ([Z_condition_gate_reg[0]] == 1) THEN Phase(0.5); PhasedX(0.5, 1.5) q[0]; Rz(1) q[0]; IF ([Z_condition_gate_reg[0]] == 1) THEN Rz(1) q[0]; IF ([tk_SCRATCH_BIT[0]] == 1) THEN Rz(3.5) q[0]; IF ([X_condition_reg[0]] == 1) THEN Rz(1) q[0]; IF ([X_condition_reg[0]] == 1) THEN PhasedX(0.5, 0.5) q[0]; Measure q[0] --> classical_reg[0]; ]

In [14]:
results = backend.get_result(handle)

In [15]:
results.get_shots()

array([[1, 1, 0, 1, 0],
       [1, 0, 1, 0, 1],
       [0, 1, 1, 1, 0],
       [1, 1, 1, 0, 1],
       [0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 1, 0, 1, 0]], dtype=uint8)

In [62]:
class_ran_bit = results.get_shots(cbits = circuit.get_c_register(name="classical_reg"))
z_res_rand_bit = results.get_shots(cbits = circuit.get_c_register(name='Z_condition_gate_reg'))
z_basis_rand_bit = results.get_shots(cbits = circuit.get_c_register(name='Z_condition_reg'))
x_basis_rand_bit = results.get_shots(cbits = circuit.get_c_register(name='X_condition_reg'))
c_res_bits = [i[0] for i in class_ran_bit]
z_res_bits = [i[0] for i in z_res_rand_bit]
z_basis_bits = [i[0] for i in z_basis_rand_bit]
x_basis_bits = [i[0] for i in x_basis_rand_bit]
sdg_column = [i & j for i,j in zip(z_basis_bits,x_basis_bits)]
basis = [hf.func_2_str_basis_letter(hf.generate_basis_from_two_bits(bit1, bit2)) for bit1,bit2 in zip(z_basis_bits,x_basis_bits)]
shot_num = [i+1 for i in range(len(c_res_bits))]

In [63]:
df = pd.DataFrame({'Shot_num':shot_num,'Z_gate_cond_bit':z_res_bits,'Z_bit_basis':z_basis_bits,'X_bit_basis':x_basis_bits,'Sdg_gate_applied':sdg_column,'H_gate_applied':x_basis_bits,'Measure_basis':basis,'Outcome':c_res_bits})

In [64]:
df.to_csv(data_name)